In [ ]:
# !pip uninstall tensorflow

In [ ]:
# !pip uninstall keras

In [ ]:
# !pip install tensorflow==1.14

In [ ]:
# !pip install keras==2.2

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout, MaxPooling1D
from keras.optimizers import SGD
from keras.utils import np_utils
import time

In [ ]:
train = pd.read_csv('/gdrive/MyDrive/데이터2/TRAIN_AB.csv')
test = pd.read_csv('/gdrive/MyDrive/데이터2/TEST_AB.csv')

In [ ]:
def encode(train, test):
    label_encoder = LabelEncoder().fit(train.activity)
    labels = label_encoder.transform(train.activity)
    test_labels = label_encoder.transform(test.activity)
    classes = list(label_encoder.classes_)

    train = train.drop(['activity'], axis=1)
    test = test.drop('activity', axis=1)

    return train, labels, test, test_labels,classes

In [ ]:
#train : label이 제거되고 데이터만 남은 train dataset
#test : label이 제거되고 데이터만 남은 test dataset
#labels : 숫자라벨링 y값으로 사용할 것들 0,1,2,3
#classes : activity 항목들 ['Badminton', 'Running', 'Standing', 'Walking']
train, labels, test, test_labels, classes = encode(train, test)

In [ ]:
# standardize train features
scaler = StandardScaler().fit(train.values)
scaled_train = scaler.transform(train.values)
scaled_test = scaler.transform(test.values)


In [ ]:
scaled_train

In [ ]:
# split train data into train and validation
sss = StratifiedShuffleSplit(test_size=0.1, random_state=23)
for train_index, valid_index in sss.split(scaled_train, labels):
    X_train, X_valid = scaled_train[train_index], scaled_train[valid_index]
    y_train, y_valid = labels[train_index], labels[valid_index]



In [ ]:
nb_features = 100 # number of features per features type (shape, texture, margin)   
nb_class = len(classes)


# reshape train data
X_train_r = np.zeros((len(X_train), nb_features, 6))
X_train_r[:, :, 0] = X_train[:, 0:nb_features]
X_train_r[:, :, 1] = X_train[:, nb_features:200]
X_train_r[:, :, 2] = X_train[:, 200:300]
X_train_r[:, :, 0] = X_train[:, 300:400]
X_train_r[:, :, 1] = X_train[:, 400:500]
X_train_r[:, :, 2] = X_train[:, 500:]

# reshape validation data
X_valid_r = np.zeros((len(X_valid), nb_features, 6))
X_valid_r[:, :, 0] = X_valid[:, :nb_features]
X_valid_r[:, :, 1] = X_valid[:, nb_features:200]
X_valid_r[:, :, 2] = X_valid[:, 200:300]
X_valid_r[:, :, 0] = X_valid[:, 300:400]
X_valid_r[:, :, 1] = X_valid[:, 400:500]
X_valid_r[:, :, 2] = X_valid[:, 500:]


# reshape validation data
X_test_r = np.zeros((len(scaled_test), nb_features, 6))
X_test_r[:, :, 0] = scaled_test[:, :nb_features]
X_test_r[:, :, 1] = scaled_test[:, nb_features:200]
X_test_r[:, :, 2] = scaled_test[:, 200:300]
X_test_r[:, :, 0] = scaled_test[:, 300:400]
X_test_r[:, :, 1] = scaled_test[:, 400:500]
X_test_r[:, :, 2] = scaled_test[:, 500:]


In [ ]:
X_test_r

In [ ]:
# Keras model with one Convolution1D layer
# unfortunately more number of covnolutional layers, filters and filters lenght 
# don't give better accuracy
model = Sequential()
model.add(Convolution1D(nb_filter=32, filter_length=10, input_shape=(nb_features, 6)))
model.add(MaxPooling1D(pool_size=2, strides=None, padding='same'))
model.add(Convolution1D(nb_filter=32, filter_length=10, input_shape=(nb_features, 6)))
model.add(MaxPooling1D(pool_size=2, strides=None, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(512, activation='relu'))
model.add(Dense(nb_class))
model.add(Activation('softmax'))
model.summary()


In [ ]:
y_train = np_utils.to_categorical(y_train, nb_class)
y_valid = np_utils.to_categorical(y_valid, nb_class)
test_labels = np_utils.to_categorical(test_labels, nb_class)

sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

nb_epoch = 15
#model.fit(X_train_r, y_train, nb_epoch=nb_epoch, validation_data=(X_valid_r, y_valid), batch_size=16)
start = time.time()
mhistory = model.fit(X_train_r, y_train, epochs=nb_epoch,validation_data=(X_valid_r, y_valid), batch_size=16)
print("time : ", time.time()-start)

In [ ]:
# 모델 평가
print('\n-- Evalutate --')
scores = model.evaluate(X_test_r, test_labels, verbose=1)

print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))


In [ ]:
import matplotlib.pyplot as plt

# 모델 저장
#model.save('/gdrive/MyDrive/데이터2/모델저장/AB/model20_5.h5')

# 학습과정 그래프
fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(mhistory.history['loss'], 'y', label='train loss')
loss_ax.plot(mhistory.history['val_loss'], 'r', label='val loss')
loss_ax.set_ylim([-0.2, 1.2])

acc_ax.plot(mhistory.history['acc'], 'b', label='train acc')
acc_ax.plot(mhistory.history['val_acc'], 'g', label='val acc')

acc_ax.set_ylim([-0.2, 1.2])


loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuracy')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

fig = plt.gcf()
plt.show()
fig.savefig('/gdrive/MyDrive/데이터2/모델그래프/AB/model30_5.png', bbox_inches='tight')
